In [1]:
from datetime import datetime
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import pandas as pd
import yahoo_fin.stock_info as si
from utils.utils_bigquery import *
from datetime import *
import talib as ta

In [2]:
key_path = key_path
project = project_id

# Sources
table = 'bronze_ticker_data'
dataset = 'bronze'

# Tables id
table_conca = f'{project}.{dataset}.{table}'

# Loading tables
table_to_save = 'silver_indicators'
dataset_to_save = 'silver'
table_conca_to_save = f'{project}.{dataset_to_save}.{table_to_save}'


In [3]:
# Conectamos con Bigquery
bigquery = BigQueryUtils(key_path)

In [4]:
df_all = bigquery.run_query(
    f"""
    SELECT
        *
    FROM {project}.{dataset}.{table}
    """
)
df_all

,date,open,high,low,close,adjclose,volume,ticker,id
0,2024-08-08 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,<NA>,LBAI,f8ee9f402d419cbcad15e1d44ea96cd4
1,2024-07-25 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,<NA>,CUBWU,f54f3e27270ca16f372ac7a267dcdcbe
2,2023-11-13 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,<NA>,WHLRL,047dd711da732a2d1637301b92e04136
3,2024-08-07 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,<NA>,LBAI,8c012e333fe3585350ad10997aea1a0e
4,2022-01-14 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,<NA>,WHLRL,eb8781a1798089ed2ff23791f2ee8a9d
...,...,...,...,...,...,...,...,...,...
9142312,2024-04-17 00:00:00+00:00,31.980000,32.490002,31.690001,32.279999,32.279999,3651900,MTCH,8e3c3b8945798f3a362b10d4ec030040
9142313,2024-05-06 00:00:00+00:00,31.900000,32.470001,31.879999,32.240002,32.240002,5944600,MTCH,ec14a5449bcc6eb94413beb378a68e5d
9142314,2024-06-28 00:00:00+00:00,30.350000,30.540001,30.059999,30.379999,30.379999,11048700,MTCH,2bdf5a4de59cab4f770946cc0df38cb4
9142315,2024-08-05 00:00:00+00:00,35.119999,35.830002,34.494999,35.660000,35.660000,7268500,MTCH,3c1eb088c89fe14b6ba669ea4bf44029


In [5]:
df_drop_nan = df_all.dropna()
df_drop_nan

,date,open,high,low,close,adjclose,volume,ticker,id
8,2023-11-08 00:00:00+00:00,2.010000,2.015000,1.960000,2.000000,2.000000,32900,VOXR,09c840d56192b2ff8426eaea6a977ca3
9,2023-12-26 00:00:00+00:00,1.850000,2.080000,1.800000,2.000000,2.000000,331900,MATH,b7baf77d49fa4107c1ce675b3739e69e
10,2016-12-13 00:00:00+00:00,2.020000,2.020000,2.000000,2.000000,2.000000,4000,QIPT,09c3b781ac8e79268b9604e817b4aeb3
11,2022-04-08 00:00:00+00:00,1.990000,2.060000,1.890000,2.000000,2.000000,556400,BKSY,5652fa01463219c400649d29f67a8cc6
12,2017-12-15 00:00:00+00:00,2.250000,2.550000,2.000000,2.000000,2.000000,65180,LFMD,4b678d8b622abca4921f6dc8d3c98d37
...,...,...,...,...,...,...,...,...,...
9142312,2024-04-17 00:00:00+00:00,31.980000,32.490002,31.690001,32.279999,32.279999,3651900,MTCH,8e3c3b8945798f3a362b10d4ec030040
9142313,2024-05-06 00:00:00+00:00,31.900000,32.470001,31.879999,32.240002,32.240002,5944600,MTCH,ec14a5449bcc6eb94413beb378a68e5d
9142314,2024-06-28 00:00:00+00:00,30.350000,30.540001,30.059999,30.379999,30.379999,11048700,MTCH,2bdf5a4de59cab4f770946cc0df38cb4
9142315,2024-08-05 00:00:00+00:00,35.119999,35.830002,34.494999,35.660000,35.660000,7268500,MTCH,3c1eb088c89fe14b6ba669ea4bf44029


In [6]:
# Obtener el año actual
año_actual = datetime.now().year

# Agrupar por el campo 'ticker' y filtrar aquellos cuyo valor máximo de 'date' sea igual al año actual
tickers_filtrados = df_drop_nan.groupby('ticker').filter(lambda x: x['date'].max().year == año_actual)

# Filtramos aquellos tickers que tengan más de 5 años
tickers_filtrados = tickers_filtrados.groupby('ticker').count()
tickers_filtrados = tickers_filtrados[tickers_filtrados['date'] >= 252 * 5]

tickers_filtrados = tickers_filtrados.reset_index()
tickers_filtrados = tickers_filtrados['ticker']

tickers_filtrados


0          A
1       AACG
2       AADI
3       AADR
4        AAL
        ... 
3440     ZUO
3441    ZVRA
3442     ZWS
3443    ZYME
3444    ZYXI
Name: ticker, Length: 3445, dtype: object

In [8]:
df = pd.merge(df_drop_nan, tickers_filtrados, how='inner', on='ticker')
df = df[df.columns]
df

,date,open,high,low,close,adjclose,volume,ticker,id
0,2023-12-26 00:00:00+00:00,1.850000,2.080000,1.800000,2.000000,2.000000,331900,MATH,b7baf77d49fa4107c1ce675b3739e69e
1,2020-11-25 00:00:00+00:00,4.500000,4.700000,4.230000,4.500000,4.500000,55400,MATH,13f8144ad221228a746b095b53ca62b2
2,2021-12-09 00:00:00+00:00,1.310000,1.310000,1.220000,1.250000,1.250000,66200,MATH,62921419e9152ed4d7fce410902c35dc
3,2022-05-05 00:00:00+00:00,1.000000,1.010000,0.950000,1.000000,1.000000,22500,MATH,0e697d21754326ccdb0ee1408c07df7b
4,2023-05-01 00:00:00+00:00,1.000000,1.000000,1.000000,1.000000,1.000000,2100,MATH,926ff6f9bd98a26e1f6bc6e2d0fb0a6d
...,...,...,...,...,...,...,...,...,...
7821003,2023-12-28 00:00:00+00:00,52.730000,53.750000,52.650002,53.669998,53.669998,2129300,TSN,ff4b946b6a21258a242f137b4e1d482f
7821004,2024-01-31 00:00:00+00:00,55.130001,55.320000,54.470001,54.759998,54.759998,4236300,TSN,8553cfacc22c81d22397a77f32085eee
7821005,2024-02-12 00:00:00+00:00,52.660000,52.990002,52.169998,52.549999,52.549999,3101200,TSN,8da7365a13f9c74fe9e3ff2b718e6b10
7821006,2024-05-01 00:00:00+00:00,60.360001,61.020000,59.980000,60.290001,60.290001,2370400,TSN,6e160a1756ee01517b9d803a202749f2


In [9]:
def calculate_indicators(df):
    # Indicadores técnicos
    df['SMA_20'] = ta.SMA(df['close'], timeperiod=20)
    df['EMA_50'] = ta.EMA(df['close'], timeperiod=50)
    df['WILLR'] = ta.WILLR(df['high'], df['low'], df['close'], timeperiod=14)
    df['ATR'] = ta.ATR(df['high'], df['low'], df['close'], timeperiod=14)
    df['ADX'] = ta.ADX(df['high'], df['low'], df['close'], timeperiod=14)
    df['CCI'] = ta.CCI(df['high'], df['low'], df['close'], timeperiod=14)
    df['OBV'] = ta.OBV(df['close'], df['volume'])
    df['RSI'] = ta.RSI(df['close'], timeperiod=14)

    # Patrones de velas
    df['CDL_DOJI'] = ta.CDLDOJI(df['open'], df['high'], df['low'], df['close'])
    df['CDL_HAMMER'] = ta.CDLHAMMER(
        df['open'], df['high'], df['low'], df['close'])
    df['CDL_MORNING_STAR'] = ta.CDLMORNINGSTAR(
        df['open'], df['high'], df['low'], df['close'])
    df['CDL_SHOOTING_STAR'] = ta.CDLSHOOTINGSTAR(
        df['open'], df['high'], df['low'], df['close'])
    df['CDL_ENGULFING'] = ta.CDLENGULFING(
        df['open'], df['high'], df['low'], df['close'])
    df['CDL_LONGLINE'] = ta.CDLLONGLINE(
        df['open'], df['high'], df['low'], df['close'])

    # Otros indicadores adicionales para medir la volatilidad
    df['bb_bbh'], df['bb_bbm'], df['bb_bbl'] = ta.BBANDS(df['close'], timeperiod=20,
                   nbdevup=2, nbdevdn=2, matype=0)
    df['daily_return'] = df['close'].pct_change()
    df['macd'], df['macd_signal'], df['macd_hist'] = ta.MACD(df['close'], slowperiod=26,
                   fastperiod=12, signalperiod=9)
    df['stoch'], df['stoch_d'] = ta.STOCH(
        high=df['high'], low=df['low'], close=df['close'], fastk_period=21, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    
    return df

In [10]:
df = calculate_indicators(df)
df

,date,open,high,low,close,adjclose,volume,ticker,id,SMA_20,...,CDL_LONGLINE,bb_bbh,bb_bbm,bb_bbl,daily_return,macd,macd_signal,macd_hist,stoch,stoch_d
0,2023-12-26 00:00:00+00:00,1.850000,2.080000,1.800000,2.000000,2.000000,331900,MATH,b7baf77d49fa4107c1ce675b3739e69e,NaN,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-11-25 00:00:00+00:00,4.500000,4.700000,4.230000,4.500000,4.500000,55400,MATH,13f8144ad221228a746b095b53ca62b2,NaN,...,0,NaN,NaN,NaN,1.250000,NaN,NaN,NaN,NaN,NaN
2,2021-12-09 00:00:00+00:00,1.310000,1.310000,1.220000,1.250000,1.250000,66200,MATH,62921419e9152ed4d7fce410902c35dc,NaN,...,0,NaN,NaN,NaN,-0.722222,NaN,NaN,NaN,NaN,NaN
3,2022-05-05 00:00:00+00:00,1.000000,1.010000,0.950000,1.000000,1.000000,22500,MATH,0e697d21754326ccdb0ee1408c07df7b,NaN,...,0,NaN,NaN,NaN,-0.200000,NaN,NaN,NaN,NaN,NaN
4,2023-05-01 00:00:00+00:00,1.000000,1.000000,1.000000,1.000000,1.000000,2100,MATH,926ff6f9bd98a26e1f6bc6e2d0fb0a6d,NaN,...,0,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7821003,2023-12-28 00:00:00+00:00,52.730000,53.750000,52.650002,53.669998,53.669998,2129300,TSN,ff4b946b6a21258a242f137b4e1d482f,66.496998,...,100,66.496998,66.496998,66.496998,0.035900,-7.764849,-6.191727,-1.573122,14.923729,9.941363
7821004,2024-01-31 00:00:00+00:00,55.130001,55.320000,54.470001,54.759998,54.759998,4236300,TSN,8553cfacc22c81d22397a77f32085eee,64.852998,...,0,64.852998,64.852998,64.852998,0.020309,-7.343572,-6.422096,-0.921476,16.501137,13.610302
7821005,2024-02-12 00:00:00+00:00,52.660000,52.990002,52.169998,52.549999,52.549999,3101200,TSN,8da7365a13f9c74fe9e3ff2b718e6b10,62.726498,...,0,62.726498,62.726498,62.726498,-0.040358,-7.106120,-6.558901,-0.547219,16.981499,16.135455
7821006,2024-05-01 00:00:00+00:00,60.360001,61.020000,59.980000,60.290001,60.290001,2370400,TSN,6e160a1756ee01517b9d803a202749f2,61.375498,...,0,61.375498,61.375498,61.375498,0.147288,-6.221665,-6.491454,0.269788,23.114939,18.865858


In [11]:
nulos_por_columna = df.isnull().sum()
nulos_por_columna

date                  0
open                  0
high                  0
low                   0
close                 0
adjclose              0
volume                0
ticker                0
id                    0
SMA_20               19
EMA_50               49
WILLR                13
ATR                  14
ADX                  27
CCI                  13
OBV                   0
RSI                  14
CDL_DOJI              0
CDL_HAMMER            0
CDL_MORNING_STAR      0
CDL_SHOOTING_STAR     0
CDL_ENGULFING         0
CDL_LONGLINE          0
bb_bbh               19
bb_bbm               19
bb_bbl               19
daily_return          1
macd                 33
macd_signal          33
macd_hist            33
stoch                24
stoch_d              24
dtype: int64

In [12]:
df = df.fillna(0)
nulos_por_columna = df.isnull().sum()
nulos_por_columna

date                 0
open                 0
high                 0
low                  0
close                0
adjclose             0
volume               0
ticker               0
id                   0
SMA_20               0
EMA_50               0
WILLR                0
ATR                  0
ADX                  0
CCI                  0
OBV                  0
RSI                  0
CDL_DOJI             0
CDL_HAMMER           0
CDL_MORNING_STAR     0
CDL_SHOOTING_STAR    0
CDL_ENGULFING        0
CDL_LONGLINE         0
bb_bbh               0
bb_bbm               0
bb_bbl               0
daily_return         0
macd                 0
macd_signal          0
macd_hist            0
stoch                0
stoch_d              0
dtype: int64

In [14]:
try:
    # Filtramos solamente los nuevos registros
    df_incremental = bigquery.select_for_incremental(id='id', table=table_conca_to_save, new_df=df)

    if not df_incremental.empty:
        # Guardamos los datos en Bigquery
        bigquery.save_dataframe(df_incremental, project, dataset_to_save, table_to_save, if_exists='append', schema=None)
        print(f'New records loaded')
    else:
        print('No new records to load.')

# En el caso de no tener datos en Bigquery, guardamos todo el df
except Exception as e:
    bigquery.save_dataframe(df, project, dataset_to_save, table_to_save, if_exists='replace', schema=None)
    print('New data persisted')
    print(f'Exception encountered: {e}')

No new records to load.
